<style>
    h1 {
    font-family: 'Segoe UI Semibold';
    font-size: 40px;
    text-align: center;
  }
</style>

# Proyecto 3: Saber 11

<style>
  .image {
            overflow: hidden;
            width: 100%;
            border-radius: 20px;
            object-fit: cover;
        }
  .container {
            width: 100%;
            height: 300px;
        }

</style>

<div class="container">
    <img src="../img/image.png" class="image"/>
</div>

<style>
  p {
    font-family: 'Segoe UI Light';
    font-size: 20px;
    text-align: center;
  }
</style>

<div style="text-align: center;">
  Analítica Computacional para la Toma de Decisiones IIND-4130 <br>
  Universidad de los Andes &copy; 2024 <br>
  19 de noviembre de 2024 <br><br>
  <strong>GRUPO 7:</strong> <br>
  Laura Calderón - 202122045 <br>
  Camilo Duque - 202024289<br>
  Daniela Espinosa - 202022615<br>
</div>


## **Tarea 2**
Limpieza de Datos

### **Proceso de Limpieza Previo en AWS Athena**

A continuación, se describe el proceso de limpieza y preparación realizado en AWS Athena para la obtención de una tabla depurada que será utilizada en el análisis:

1. **Filtrado por Departamento:**
   - Se seleccionaron únicamente los registros correspondientes a colegios ubicados en el departamento de **Boyacá**, filtrando la tabla según el campo `cole_depto_ubicacion`.

2. **Eliminación de Registros Duplicados:**
   - Se utilizó la cláusula `DISTINCT` para eliminar registros duplicados de la tabla, asegurando que cada fila representara datos únicos.

3. **Manejo de Datos Faltantes:**
   - Se observó que para los datos correspondientes a los periodos entre `20101` y `20141`, no se disponía de registros sobre los puntajes obtenidos por los estudiantes en las pruebas ICFES. Esto representaba un **43% de datos faltantes** en las columnas relacionadas con los puntajes.
   - Dado que estos registros no aportaban valor al análisis, se decidió **eliminarlos**. Por lo tanto, el estudio se enfoca únicamente en datos desde el periodo `20142` en adelante.

4. **Eliminación de Columnas Redundantes o Poco Informativas:**
   - **`cole_cod_depto_ubicacion`:** Representa únicamente el identificador del departamento (15 para Boyacá). Dado que todos los registros pertenecen al mismo departamento, esta columna se eliminó.
   - **`estu_consecutivo`:** Se verificó que este campo, utilizado como identificador único del estudiante, no tuviera valores duplicados. Tras confirmar su unicidad, se eliminó al no ser relevante para el análisis.
   - **`cole_depto_ubicacion`:** Esta columna también se eliminó, ya que toda la tabla corresponde al departamento de Boyacá y, por ende, no añade información adicional.
   - **`cole_cod_mcpio_ubicacion`:** Este campo numérico representaba el municipio del colegio. Sin embargo, su información es redundante con la columna `cole_mcpio_ubicacion`, que contiene el nombre del municipio. Por lo tanto, se eliminó.
   - **`estu_cod_depto_presentacion` y `estu_cod_mcpio_presentacion`:** Ambas columnas contienen códigos numéricos que representan los departamentos y municipios de presentación del estudiante, pero la misma información ya está disponible en `estu_depto_presentacion` y `estu_mcpio_presentacion`. Por esta razón, se eliminaron.
   - **`estu_cod_reside_depto` y `estu_cod_reside_mcpio`:** Similar a los casos anteriores, estas columnas contienen códigos de los departamentos y municipios de residencia, pero la misma información ya está en `estu_depto_reside` y `estu_mcpio_reside`. Se procedió a eliminarlas.
   - **`cole_cod_dane_establecimiento` y `cole_cod_dane_sede`:** Estas columnas contienen identificadores únicos de los colegios y sedes, pero no aportan valor al análisis y, por ende, se eliminaron.

5. **Resultado Final:**
   - El resultado del proceso es una tabla depurada en AWS Athena. Esta tabla se exportó en formato CSV y será utilizada para continuar con el proceso de limpieza y análisis en este notebook.


<style>
  .image {
            overflow: hidden;
            width: 100%;
            border-radius: 20px;
            object-fit: cover;
        }
  .container {
            width: 100%;
            height: 750px;
        }

</style>

<div class="container">
    <img src="../img/Consulta_Final.png" class="image"/>
</div>

In [77]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from scipy.stats import zscore
from sklearn.preprocessing import OneHotEncoder

dta = pd.read_csv('../data/Datos_Boyaca.csv')

# Mostrar información básica del DataFrame
print(f"Total de registros: {dta.shape[0]}")
print(f"Total de columnas: {dta.shape[1]}")

# Mostrar un resumen de las columnas
print("\nResumen de columnas:")
print(dta.info())

Total de registros: 100049
Total de columnas: 41

Resumen de columnas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100049 entries, 0 to 100048
Data columns (total 41 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   periodo                      100049 non-null  int64  
 1   estu_tipodocumento           100049 non-null  object 
 2   cole_area_ubicacion          100049 non-null  object 
 3   cole_bilingue                91518 non-null   object 
 4   cole_calendario              100049 non-null  object 
 5   cole_caracter                99600 non-null   object 
 6   cole_codigo_icfes            100049 non-null  int64  
 7   cole_genero                  100049 non-null  object 
 8   cole_jornada                 100049 non-null  object 
 9   cole_mcpio_ubicacion         100049 non-null  object 
 10  cole_naturaleza              100049 non-null  object 
 11  cole_nombre_establecimiento  100049 non-null  

In [78]:
# Verificar que no haya filas duplicadas
duplicados = dta.duplicated()

# Contar cuántas filas duplicadas hay
num_duplicados = duplicados.sum()
print(f"Número de filas duplicadas: {num_duplicados}")

Número de filas duplicadas: 0


In [79]:
# Calcular la cantidad de valores faltantes y el porcentaje
faltantes = dta.isnull().sum()
porcentaje_faltantes = (dta.isnull().sum() / len(dta)) * 100

resumen_faltantes = pd.DataFrame({
    'Valores Faltantes': faltantes,
    'Porcentaje (%)': porcentaje_faltantes
})

# Filtrar para mostrar solo las columnas con valores faltantes
resumen_faltantes = resumen_faltantes[resumen_faltantes['Valores Faltantes'] > 0]
resumen_faltantes = resumen_faltantes.sort_values(by='Porcentaje (%)', ascending=False)

print(resumen_faltantes)

                      Valores Faltantes  Porcentaje (%)
cole_bilingue                      8531        8.526822
fami_estratovivienda               2642        2.640706
fami_tieneinternet                 2464        2.462793
fami_educacionmadre                2382        2.380833
fami_educacionpadre                2381        2.379834
fami_personashogar                 1598        1.597217
fami_cuartoshogar                  1541        1.540245
fami_tieneautomovil                1540        1.539246
fami_tienelavadora                 1491        1.490270
fami_tienecomputador               1489        1.488271
cole_caracter                       449        0.448780
estu_genero                         115        0.114944
estu_depto_reside                    62        0.061970
estu_mcpio_reside                    62        0.061970
desemp_ingles                         2        0.001999
punt_ingles                           2        0.001999


In [80]:
# Crear un diccionario para almacenar la proporción máxima y el valor más repetido por columna
columnas_dominantes = {}

for columna in dta.columns:
    valor_mas_comun = dta[columna].value_counts(normalize=True, dropna=False).idxmax()
    proporción_max = dta[columna].value_counts(normalize=True, dropna=False).max()
    columnas_dominantes[columna] = {'Proporción Max (%)': proporción_max * 100, 'Valor Más Común': valor_mas_comun}

resultado = pd.DataFrame.from_dict(columnas_dominantes, orient='index')
columnas_repetitivas = resultado[resultado['Proporción Max (%)'] >= 95]
columnas_repetitivas = columnas_repetitivas.sort_values(by='Proporción Max (%)', ascending=False)

print("Columnas con 95% o más de valores repetidos:")
print(columnas_repetitivas)


Columnas con 95% o más de valores repetidos:
                          Proporción Max (%) Valor Más Común
cole_calendario                   100.000000               A
estu_estudiante                   100.000000      ESTUDIANTE
estu_privado_libertad              99.947026               N
estu_estadoinvestigacion           99.917041        PUBLICAR
estu_nacionalidad                  99.909045        COLOMBIA
estu_pais_reside                   99.909045        COLOMBIA
estu_depto_reside                  99.371308          BOYACA
estu_depto_presentacion            98.460754          BOYACA
cole_genero                        97.716119           MIXTO
cole_sede_principal                97.330308               S


In [81]:
# Identificar columnas con más del 95% de valores repetidos y eliminnarlas
columnas_a_eliminar = columnas_repetitivas.index.tolist()
dta= dta.drop(columns=columnas_a_eliminar)

In [82]:
# Identificar columnas numéricas en el DataFrame
columnas_numericas = dta.select_dtypes(include=['number']).columns
print("Columnas numéricas:")
print(columnas_numericas)

if len(columnas_numericas) == 0:
    print("No hay columnas numéricas en el DataFrame.")
else:
    print(f"Total de columnas numéricas: {len(columnas_numericas)}")

Columnas numéricas:
Index(['periodo', 'cole_codigo_icfes', 'punt_ingles', 'punt_matematicas',
       'punt_sociales_ciudadanas', 'punt_c_naturales', 'punt_lectura_critica',
       'punt_global'],
      dtype='object')
Total de columnas numéricas: 8


In [83]:
# Identificar las columnas categóricas y las columnas numéricas con faltantes
columnas_categoricas = [
    'cole_bilingue', 'fami_estratovivienda', 'fami_tieneinternet', 
    'fami_educacionmadre', 'fami_educacionpadre', 'fami_tieneautomovil',
    'fami_tienelavadora', 'fami_tienecomputador', 'cole_caracter', 
    'estu_genero','fami_personashogar', 'fami_cuartoshogar','desemp_ingles','estu_cod_reside_mcpio','estu_mcpio_reside'
]

columnas_numericas_faltantes = [
    'punt_ingles', 'punt_global'
]

print("\nSe imputaan valores faltante en columnas categóricas con la moda:")
# Imputar valores faltantes en columnas categóricas con la moda
for col in columnas_categoricas:
    if col in dta.columns:
        moda = dta[col].mode()[0]
        dta[col] = dta[col].fillna(moda)
        print(f" '{col}':  {moda}")

print("\nSe imputaan valores faltante en columnas numéricas con la media:")
# Imputar valores faltantes en columnas numéricas con la media
for col in columnas_numericas_faltantes:
    if col in dta.columns:
        media = dta[col].mean()
        dta[col] = dta[col].fillna(media)
        print(f" '{col}': {media}")

# Verificar que no queden valores faltantes
faltantes_restantes = dta.isnull().sum()
print("\nValores faltantes restantes por columna después de la imputación:")
print(faltantes_restantes[faltantes_restantes > 0])


Se imputaan valores faltante en columnas categóricas con la moda:
 'cole_bilingue':  N
 'fami_estratovivienda':  Estrato 2
 'fami_tieneinternet':  No
 'fami_educacionmadre':  Secundaria (Bachillerato) completa
 'fami_educacionpadre':  Primaria incompleta
 'fami_tieneautomovil':  No
 'fami_tienelavadora':  Si
 'fami_tienecomputador':  Si
 'cole_caracter':  TÉCNICO/ACADÉMICO
 'estu_genero':  F
 'fami_personashogar':  3 a 4
 'fami_cuartoshogar':  Tres
 'desemp_ingles':  A-
 'estu_mcpio_reside':  TUNJA

Se imputaan valores faltante en columnas numéricas con la media:
 'punt_ingles': 51.63191300088958
 'punt_global': 264.74132674989255

Valores faltantes restantes por columna después de la imputación:
Series([], dtype: int64)


### **Limpieza Final de acuerdo a las preguntas de negocio y al plan de acción**

#### **1. ¿Cuáles factores sociodemográficos están más relacionados con el mal desempeño de los estudiantes de colegios de Boyacá en la prueba Saber 11?**
**Columnas relevantes:**
- `fami_estratovivienda` (estrato socioeconómico).
- `fami_educacionmadre` y `fami_educacionpadre` (nivel educativo de los padres).
- `fami_tieneinternet`, `fami_tienecomputador`, `fami_tienelavadora`, `fami_tieneautomovil` (accesos familiares).
- `estu_genero` (género).
- `estu_mcpio_reside` (municipio de residencia).
- `fami_personashogar` y `fami_cuartoshogar` (tamaño del hogar).
- **`periodo`** (para analizar si hay tendencias en el tiempo relacionadas con los factores sociodemográficos).

---

#### **2. ¿¿Qué cambios pueden implementar los directivos de las instituciones educativas en Boyacá para aumentar el puntaje global del colegio en la Prueba Saber 11? ?**
**Columnas relevantes:**
- `cole_area_ubicacion` (rural o urbano).
- `cole_bilingue` (si es bilingüe).
- `cole_caracter` (público/privado).
- `cole_naturaleza` (carácter académico o técnico).
- `cole_jornada` (jornada escolar).
- **`periodo`** (para evaluar si el impacto de estas características ha cambiado en diferentes periodos).

---

  
### **Columnas Eliminadas y Justificación**

1. **Columnas relacionadas con identificadores o información redundante:**
   - `estu_tipodocumento`, `cole_codigo_icfes`, `cole_nombre_establecimiento`, `cole_nombre_sede`, `cole_sede_principal`:
     - Estas columnas contienen información de identificación o etiquetas de colegios y estudiantes que no aportan al análisis del desempeño ni a las preguntas planteadas.

2. **Columnas relacionadas con localización no utilizadas:**
   - `cole_mcpio_ubicacion`, `estu_mcpio_presentacion`:
     - Si bien podrían ser útiles en otros contextos, estas columnas no se alinean directamente con las preguntas seleccionadas. Se priorizó `estu_mcpio_reside` como la variable clave para analizar la ubicación de residencia.

3. **Puntajes específicos de áreas (redundantes)::**
   - `punt_ingles`, `punt_matematicas`,`punt_ingles`, `punt_sociales_ciudadanas`,`punt_c_naturales`,`punt_lectura_critica`:
     - Estos ya están reflejados en punt_global, que es la suma ponderada de estas áreas. Dado que el puntaje global es la variable objetivo, los puntajes específicos se pueden omitir.
  


In [84]:
# Lista de columnas a eliminar
columnas_a_eliminar = [
    'estu_tipodocumento',
    'cole_codigo_icfes', 'cole_nombre_establecimiento', 'cole_nombre_sede','estu_fechanacimiento'
    'cole_sede_principal', 'cole_mcpio_ubicacion', 'estu_mcpio_presentacion','punt_ingles', 'desemp_ingles','punt_matematicas','punt_ingles', 'punt_sociales_ciudadanas','punt_c_naturales','punt_lectura_critica'
]

# Eliminar las columnas seleccionadas
dta = dta.drop(columns=columnas_a_eliminar, errors='ignore')


In [85]:
# Mostrar información básica del DataFrame
print(f"Total de registros: {dta.shape[0]}")
print(f"Total de columnas: {dta.shape[1]}")

print(dta.info())

Total de registros: 100049
Total de columnas: 19
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100049 entries, 0 to 100048
Data columns (total 19 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   periodo               100049 non-null  int64 
 1   cole_area_ubicacion   100049 non-null  object
 2   cole_bilingue         100049 non-null  object
 3   cole_caracter         100049 non-null  object
 4   cole_jornada          100049 non-null  object
 5   cole_naturaleza       100049 non-null  object
 6   estu_fechanacimiento  100049 non-null  object
 7   estu_genero           100049 non-null  object
 8   estu_mcpio_reside     100049 non-null  object
 9   fami_cuartoshogar     100049 non-null  object
 10  fami_educacionmadre   100049 non-null  object
 11  fami_educacionpadre   100049 non-null  object
 12  fami_estratovivienda  100049 non-null  object
 13  fami_personashogar    100049 non-null  object
 14  fami_tieneautomovil

In [86]:
columnas_numericas = dta.select_dtypes(include=['number']).columns

# Mostrar los valores mínimos y máximos para las columnas numéricas
for col in columnas_numericas:
    if col in dta.columns:
        print(f"Columna '{col}': Min = {dta[col].min()}, Max = {dta[col].max()}")

Columna 'periodo': Min = 20142, Max = 20224
Columna 'punt_global': Min = 0, Max = 471


In [87]:
# Seleccionar columnas no numéricas
columnas_no_numericas = dta.select_dtypes(include=['object']).columns

label_encoders = {}
mapeos = {}

for col in columnas_no_numericas:
    le = LabelEncoder()
    dta[col] = le.fit_transform(dta[col].astype(str))
    label_encoders[col] = le
    mapeos[col] = dict(zip(le.classes_, le.transform(le.classes_)))


for col, mapeo in mapeos.items():
    print(f"Columna: {col}")
    for original, transformado in mapeo.items():
        print(f"  {original} -> {transformado}")
    print("-" * 30)

Columna: cole_area_ubicacion
  RURAL -> 0
  URBANO -> 1
------------------------------
Columna: cole_bilingue
  N -> 0
  S -> 1
------------------------------
Columna: cole_caracter
  ACADÉMICO -> 0
  NO APLICA -> 1
  TÉCNICO -> 2
  TÉCNICO/ACADÉMICO -> 3
------------------------------
Columna: cole_jornada
  COMPLETA -> 0
  MAÑANA -> 1
  NOCHE -> 2
  SABATINA -> 3
  TARDE -> 4
  UNICA -> 5
------------------------------
Columna: cole_naturaleza
  NO OFICIAL -> 0
  OFICIAL -> 1
------------------------------
Columna: estu_fechanacimiento
  01/01/1900 -> 0
  01/01/1969 -> 1
  01/01/1975 -> 2
  01/01/1987 -> 3
  01/01/1989 -> 4
  01/01/1993 -> 5
  01/01/1994 -> 6
  01/01/1995 -> 7
  01/01/1996 -> 8
  01/01/1997 -> 9
  01/01/1998 -> 10
  01/01/1999 -> 11
  01/01/2000 -> 12
  01/01/2001 -> 13
  01/01/2002 -> 14
  01/01/2003 -> 15
  01/01/2004 -> 16
  01/01/2005 -> 17
  01/01/2006 -> 18
  01/02/1967 -> 19
  01/02/1976 -> 20
  01/02/1982 -> 21
  01/02/1987 -> 22
  01/02/1992 -> 23
  01/02/19

In [88]:
for col in columnas_no_numericas:
    print(f"Frecuencia de valores en '{col}':")
    print(dta[col].value_counts())
    print("-" * 30)

Frecuencia de valores en 'cole_area_ubicacion':
cole_area_ubicacion
1    84547
0    15502
Name: count, dtype: int64
------------------------------
Frecuencia de valores en 'cole_bilingue':
cole_bilingue
0    98927
1     1122
Name: count, dtype: int64
------------------------------
Frecuencia de valores en 'cole_caracter':
cole_caracter
3    45951
0    32206
2    21246
1      646
Name: count, dtype: int64
------------------------------
Frecuencia de valores en 'cole_jornada':
cole_jornada
0    48411
1    34599
5     6354
4     4745
3     4171
2     1769
Name: count, dtype: int64
------------------------------
Frecuencia de valores en 'cole_naturaleza':
cole_naturaleza
1    81513
0    18536
Name: count, dtype: int64
------------------------------
Frecuencia de valores en 'estu_fechanacimiento':
estu_fechanacimiento
4871    66
6145    63
6385    63
3057    62
5092    62
        ..
4099     1
1960     1
6740     1
3391     1
1795     1
Name: count, Length: 6758, dtype: int64
--------------

## **Tarea 3**
Exploración de Datos

## **Tarea 4**
Modelamiento


In [96]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Periodo es la única columna a normalizar
columnas_a_normalizar = [
     'periodo'
]

scaler = MinMaxScaler()
dta[columnas_a_normalizar] = scaler.fit_transform(dta[columnas_a_normalizar])



#### **Primer Modelo: Red neuronal básica (Regresión)**
Características:
- Dos capas ocultas.
- Sin regularización.

In [90]:
def modelo_basico(X_train):
    model = Sequential([
        Dense(64, input_dim=X_train.shape[1], activation='relu'),
        Dense(32, activation='relu'),
        Dense(1)  # Predicción de puntaje global
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model


#### **Segundo Modelo:Red con más capas (Regresión Profunda)**
Características:
- Cuatro capas ocultas.
- Más neuronas por capa.
- Sin regularización.

In [91]:
def modelo_profundidad(X_train):
    model = Sequential([
        Dense(128, input_dim=X_train.shape[1], activation='relu'),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(1)  # Predicción de puntaje global
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model


#### **Tercer Modelo:Red con Dropout**
Características:
- Dos capas ocultas.
- Incluye Dropout para prevenir sobreajuste.

In [92]:
from tensorflow.keras.layers import Dropout

def modelo_con_dropout(X_train):
    model = Sequential([
        Dense(64, input_dim=X_train.shape[1], activation='relu'),
        Dropout(0.2), 
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model


#### **Cuarto Modelo: Red con Regularización L2**
Características:
- Dos capas ocultas.
- Regularización L2 para penalizar pesos grandes.

In [93]:
from tensorflow.keras.regularizers import l2

def modelo_con_l2(X_train):
    model = Sequential([
        Dense(64, input_dim=X_train.shape[1], activation='relu', kernel_regularizer=l2(0.01)),
        Dense(32, activation='relu', kernel_regularizer=l2(0.01)),
        Dense(1)  # Predicción de puntaje global
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model


#### **Quinto Modelo: Clasificación por rangos**
Características:
- Transformar punt_global en categorías.
- Clasificación con salida softmax.

In [94]:
from tensorflow.keras.utils import to_categorical

# Transformar punt_global a categorías
def transformar_a_categorias(y):
    # Crear rangos: Bajo (0-200), Medio (201-350), Alto (351-471)
    categorias = pd.cut(y, bins=[0, 200, 350, 471], labels=[0, 1, 2], include_lowest=True)
    return to_categorical(categorias)

def modelo_clasificacion(X_train):
    model = Sequential([
        Dense(64, input_dim=X_train.shape[1], activation='relu'),
        Dense(32, activation='relu'),
        Dense(3, activation='softmax')  # Tres categorías
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [95]:
from tensorflow.keras.optimizers import SGD, RMSprop

# Cambiar el optimizador
def modelo_con_sgd(X_train):
    model = Sequential([
        Dense(64, input_dim=X_train.shape[1], activation='relu'),
        Dense(32, activation='relu'),
        Dense(1) 
    ])
    model.compile(optimizer=SGD(learning_rate=0.01), loss='mse', metrics=['mae'])
    return model


In [97]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Dividir los datos
X = dta.drop(columns=['punt_global'])
y = dta['punt_global']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Función para entrenar y evaluar un modelo
def entrenar_y_evaluar(model, X_train, X_test, y_train, y_test):
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return mse, r2, history

# Entrenar cada modelo
modelos = {
    "Básico": modelo_basico(X_train),
    "Profundo": modelo_profundidad(X_train),
    "Dropout": modelo_con_dropout(X_train),
    "L2 Regularización": modelo_con_l2(X_train),
}

resultados = {}

for nombre, modelo in modelos.items():
    mse, r2, _ = entrenar_y_evaluar(modelo, X_train, X_test, y_train, y_test)
    resultados[nombre] = {"MSE": mse, "R2": r2}

print("\nResultados:")
for nombre, resultado in resultados.items():
    print(f"{nombre}: MSE={resultado['MSE']:.4f}, R2={resultado['R2']:.4f}")


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
2001/2001 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 11811.7588 - mae: 78.9240 - val_loss: 3751.2747 - val_mae: 49.0917
Epoch 2/50
2001/2001 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 3292.0977 - mae: 45.4459 - val_loss: 2888.9531 - val_mae: 42.6604
Epoch 3/50
2001/2001 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 3026.6816 - mae: 43.5149 - val_loss: 2736.2371 - val_mae: 41.6093
Epoch 4/50
2001/2001 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 2878.0513 - mae: 42.4472 - val_loss: 2757.8799 - val_mae: 41.8118
Epoch 5/50
2001/2001 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 2743.9470 - mae: 41.5863 - val_loss: 2801.5991 - val_mae: 42.1782
Epoch 6/50
2001/2001 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 2531.0200 - mae: 39.9938 - val_loss: 2613.0447 - val_mae: 40.7875
Epoch 7/50
2001/2001 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 2499.5989 - mae: 39.7267 - val_loss: 2383.0789 - val_mae: 38.8924
Epoch 8/50
2001/2001 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 2366.7876 - mae: 38.7512 - val_loss: 2265